<a href="https://colab.research.google.com/github/Tar-ive/txst-ai/blob/master/txst_agentic_rag_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai chromadb tiktoken python-dotenv bs4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 11.3 MB/s eta 0:00:0

In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [5]:
import os
from dotenv import load_dotenv
import openai

from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.agents import AgentExecutor, Tool, initialize_agent
from langchain.agents import AgentType
from langchain.tools import tool
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory

from bs4 import BeautifulSoup


In [6]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [10]:
openai_api_key = userdata.get('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OpenAI API key not found in environment variables.")

In [15]:
%pip install -qU langchain-openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00


In [22]:
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI # Import the ChatOpenAI class from langchain.chat_models


openai_api_key = userdata.get('OPENAI_API_KEY')

# Check if the API key is valid
if not openai_api_key:
    raise ValueError("OpenAI API key not found in userdata.")

# Initialize the OpenAI LLM with the API key
llm = ChatOpenAI(
    # Pass the api_key directly to the OpenAI class
    openai_api_key=openai_api_key,
    model_name="gpt-4o-mini", #Changed to the correct name for gpt-4-turbo
    temperature=0,  # Set to 0 for deterministic outputs
)

<ipython-input-22-bd0164c0523d>:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [23]:
template = "Answer the following question as accurately as possible. If you do not know the answer, simply say you do not know.\n\nQuestion: {query}\nAnswer:"
prompt = PromptTemplate(template=template, input_variables=["query"])


In [24]:
chain = LLMChain(llm=llm, prompt=prompt)


In [25]:
response = chain.run({"query": "What sport is played at the US Open?"})
print(response)


The sport played at the US Open is tennis.


In [26]:
response = chain.run({"query": "Where was the 2024 US Open Tennis Championship held?"})
print(response)


I do not know.


In [28]:
urls = [
    "http://mycatalog.txstate.edu/undergraduate/majors/#A",
    "http://mycatalog.txstate.edu/undergraduate/majors/#B",
    "http://mycatalog.txstate.edu/undergraduate/majors/#C",
    "http://mycatalog.txstate.edu/undergraduate/majors/#D",
    "http://mycatalog.txstate.edu/undergraduate/majors/#E",
    "http://mycatalog.txstate.edu/undergraduate/majors/#F",
    "http://mycatalog.txstate.edu/undergraduate/majors/#G",
    "http://mycatalog.txstate.edu/undergraduate/majors/#H",
    "http://mycatalog.txstate.edu/undergraduate/majors/#I",
    "http://mycatalog.txstate.edu/undergraduate/majors/#J",
    "http://mycatalog.txstate.edu/undergraduate/majors/#K",
    "http://mycatalog.txstate.edu/undergraduate/majors/#L",
    "http://mycatalog.txstate.edu/undergraduate/majors/#M",
    "http://mycatalog.txstate.edu/undergraduate/majors/#N",
    "http://mycatalog.txstate.edu/undergraduate/majors/#O",
    "http://mycatalog.txstate.edu/undergraduate/majors/#P",
    "http://mycatalog.txstate.edu/undergraduate/majors/#Q",
    "http://mycatalog.txstate.edu/undergraduate/majors/#R",
    "http://mycatalog.txstate.edu/undergraduate/majors/#S",
    "http://mycatalog.txstate.edu/undergraduate/majors/#T",
    "http://mycatalog.txstate.edu/undergraduate/majors/#U",
    "http://mycatalog.txstate.edu/undergraduate/majors/#V",
    "http://mycatalog.txstate.edu/undergraduate/majors/#W",
    "http://mycatalog.txstate.edu/undergraduate/majors/#X",
    "http://mycatalog.txstate.edu/undergraduate/majors/#Y",
    "http://mycatalog.txstate.edu/undergraduate/majors/#Z",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/accounting/accounting-bba/",
    "https://mycatalog.txstate.edu/graduate/mccoy-business-administration/accounting/accounting-bba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/advertising-mass-communication-sports-media-concentration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/advertising-mass-communication-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/agriculturalsciences/agriculture-business-management-agribusiness-specialization-bsag/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/agriculturalsciences/agriculture-agricultural-mechanics-concentration/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/agriculturalsciences/agriculture-agricultural-horticulture-concentration/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/agriculturalsciences/agriculture-teacher-certification-science-technology-grades-612-bsag/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/agriculturalsciences/agriculture-bsag/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/agriculturalsciences/agriculture-animal-science-preveterinary-concentration/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/agriculturalsciences/agriculture-animal-science/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/anthropology/anthropology-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/anthropology/anthropology-bs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/organization-workforce-leadership-studies/applied-sciences-baas/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/mathematics/applied-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/sociology/applied-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/biology/aquatic-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/art-design/art-history-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/art-design/art-ba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/chemistry-biochemistry/biochemistryacs-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/chemistry-biochemistry/biochemistry-early-entry-combined-program-bs-ms/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/chemistry-biochemistry/biochemistry-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/biology/biology-teacher-certification-life-sciences-grades-7-12-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/biology/biology-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/chemistry-biochemistry/chemistry-early-entry-combined-program-bs-ms/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/chemistry-biochemistry/chemistry-teacher-certification-grades-7-12-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/chemistry-biochemistry/chemistry-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/civil-engineering-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/art-design/communication-design-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/communication-disorders/bscd/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/studies/persuasive-communication-studies-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/studies/organizational-communication-studies-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/studies/interpersonal-communication-studies-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/studies/communication-studies-teacher-certification-speech-grades-712-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/studies/communication-studies-ba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/computer-information-systems-quantitative-methods/computer-information-systems-bba/cis-business-analytics/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/computer-information-systems-quantitative-methods/cis-info-security-bba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/computer-information-systems-quantitative-methods/cis-software-dev-bba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/computer-information-systems-quantitative-methods/computer-information-systems-bba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/computer/computer-science-concentration-engineering-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/computer/computer-science-ba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/computer/computer-science-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/concrete-industry-management-minor-business-administration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/construction-science-management-residential-construction-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/construction-science-management-minor-business-administration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/consumer-affairs-family-sciences-option-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/consumer-affairs-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/consumer-affairs-teacher-certification-family-sciences-grades-612-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/criminal-justice/criminal-justice-bscj/",
    "http://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/dance-studies-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/dance-performance-choreography-emphasis-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/dance-single-field-teaching-certification-grades-812-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/dance-two-field-teaching-certification-grades-812-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/dance-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/digital-mass-communication-sports-media-concentration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/digital-mass-communication-bs/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/finance-economics/economics-ba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/finance-economics/economics-bba/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-bilingual-biliteracy-teacher-certficiation-ec6-bilingual-spanish-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-elementary-education-teacher-certficiation-ec6-esl-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-middle-teacher-certification-48-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-middle-teacher-certification-eng-langarts-reading-ss-48-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-middle-teacher-certification-math-science-48-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-middle-teacher-certification-math-48-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-middle-teacher-certification-science-48-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-ba/",
        "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-secondary-education-teacher-certification-double-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/curriculum-instruction/education-special-education-teacher-certification-special-education-ec12-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/electrical-engineering-computer-specialization-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/electrical-engineering-micro-nano-devices-systems-specialization-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/electrical-engineering-networks-communication-systems-specialization-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/electronic-media-mass-communication-sports-media-concentration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/electronic-media-mass-communication-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/engineering-technology-civil-specialization-bst/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/electrical-specialization-bst/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/engineering-technology-environmental-specialization-bst/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/engineering-technology-manufacturing-specialization-bst/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/technology/engineering-technology-mechanical-specialization-bst/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/english/creative-writing-emphasis-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/english/film-emphasis-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/english/english-secondary-tcert-double-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/english/writing-rhetoric-emphasis-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/english/english-ba/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/exercise-sports-science-health-wellness-promotion-clinical-populations-concentration-bess/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/exercise-sports-science-prerehab-sciences/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/exercise-sports-science-teacher-certification-physical-education-grades-ec12-bess/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/exercise-sports-sci-athletic-trainig-bessms/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/fashion-merchandising-plan-presentation-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/fashion-merchandising-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/finance-economics/finance-bba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/french-secondary-tcert-double-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/french-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/geography/geographic-information-science-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/geography/resource-environmental-studies-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/geography/urban-regional-planning-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/geography/water-resources-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/geography/geography-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/german-secondary-tcert-double-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/german-ba/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/health-fitness-management-business-administration-minor-bess/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/information-management/bshim/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-fine-motor-therapy-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-gross-motor-therapy-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-prechiropractic-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-precomm-disorders-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-preclinical-lab-science-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-prenursing-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-preradiation-therapy-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-prerespiratory-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/health-sciences-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/administration/healthcare-administration-bha/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/history/history-secondary-tcert-double-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/history/history-secondary-tcert-ss-double-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/history/history-two-fields-teacher-certification-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/history/history-ba/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/human-dev-family-sciences-teacher-certification-human-studies-grades-812-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/human-dev-family-sciences-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/geography/geography-ba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/industrial-engineering-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/counseling-leadership-adult-school-psychology/integrated-studies-bgs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/interior-design-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/relations-bais/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/asian-focus-bais/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/european-focus-bais/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/business-focus-bais/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/interamerican-focus-bais/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/middle-east-african-focus-bais/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/russian-east-european-focus-bais/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/international-studies/travel-tourism-focus-bais/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/journalism-mass-communication-sports-media-concentration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/journalism-mass-communication-bs/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/management/management-entrepreneurial-studies-concentration-bba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/management/management-human-resource-concentration-bba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/management/management-bba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/manufacturing-engineering-mechanical-systems-concentration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/manufacturing-engineering-smart-concentration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/marketing/marketing-professional-sales-concentration-bba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/marketing/marketing-services-concentration-bba/",
    "https://mycatalog.txstate.edu/undergraduate/mccoy-business-administration/marketing/marketing-bba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/mass-communication-ba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/mathematics/mathematics-teacher-certification-grades-7-12-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/mathematics/mathematics-ba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/mathematics/mathematics-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/ingram-school/mechanical-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/medical-laboratory-science-program/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/biology/microbiology-molecular-genetics-bs/",
    "https://next.mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/music-studies-band-concentration-teacher-certification-grades-ec12-BM/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/music-studies-choral-concentration-teacher-certification-grades-ec12-bm/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/music-studies-mariachi-concentration-teacher-certification-grades-ec12-bm/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/music-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/musical-theatre-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/st-davids-nursing/rn-to-bsn/",
    "https://mycatalog.txstate.edu/graduate/health-professions/st-davids-nursing/leadershipandadminnursing-rn-bsn-msn/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/st-davids-nursing/bsn/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/nutrition-foods-dietetics-track-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/nutrition-foods-teacher-certification-hospitality-nutrition-food-sciences-grades-812-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/family-consumer-sciences/nutrition-foods-bsfcs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/performance-composition-concentration-bm/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/performance-guitar-concentration-bm/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/performance-instrumental-concentration-bm/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/performance-jazz-concentration-bm/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/performance-keyboard-concentration-bm/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/performance-vocal-concentration-bm/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/philosophy/philosophy-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/art-design/photography-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/geography/physical-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/physics/physics-astronomy-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/physics/physics-teacher-certification-physics-math-grades-7-12-bs/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/physics/physics-ba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/physics/physics-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/political-science/political-science-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/psychology/psychology-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/psychology/psychology-science-minor-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/political-science/public-administration-bpa/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/public-health-applied-epidemiology-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/public-health-health-wellness-coaching-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/public-health-health-equity-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/public-health-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/public-relations-mass-communication-sports-media-concentration-bs/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/journalism-mass/public-relations-mass-communication-bs/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/radiation-therapy-program/bsrt/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/recreation-studies-community-recreation-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/recreation-studies-outdoor-recreation-bs/",
    "https://mycatalog.txstate.edu/undergraduate/education/health-human-performance/recreation-studies-therapeutic-recreation-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/philosophy/religious-studies-ba/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/respiratory-care/rrt-to-bsrc/",
    "https://mycatalog.txstate.edu/undergraduate/health-professions/respiratory-care/bsrc/",
    "https://mycatalog.txstate.edu/undergraduate/applied-arts/social-work/social-work-bsw/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/sociology/sociology-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/music/sound-recording-technology-bs/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/spanish-hispanic-litandculture-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/spanish-secondary-tcert-double-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/spanish-profesions-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/spanish-linguistics-ba/",
    "https://mycatalog.txstate.edu/undergraduate/liberal-arts/world-languages-literatures/spanish-ba/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/art-design/studio-art-teacher-certification-art-grades-ec12-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/art-design/studio-art-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/theatre-acting-preprofessional-option-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/theatre-film-production-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/theatre-performance-production-preprofessional-option-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/theatre-teacher-certification-grades-ec12-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/theatre-technical-production-bfa/",
    "https://mycatalog.txstate.edu/undergraduate/fine-arts-communication/theatre/theatre-ba/",
    "https://mycatalog.txstate.edu/undergraduate/science-engineering/biology/wildlife-certification-biologist-bs/"
]




In [29]:
loaders = [WebBaseLoader(url) for url in urls]
docs = []
for loader in loaders:
    loaded_docs = loader.load()
    docs.extend(loaded_docs)


In [30]:
print(docs[0].page_content[:500])  # Print the first 500 characters






Undergraduate Degree Programs | Texas State University


























Skip to Content
AZ Index
Catalog Home
Institution Home








Search Catalog

Search



HomeUndergraduateUndergraduate Degree Programs


Catalogs 2024-2025

Catalog Home
Undergraduate Degree Programs
Graduate Degree Programs

Previous Catalogs

Undergraduate
Graduate





BobcatMail
CatsWeb
Canvas



More Tools 



SAP Portal
Pay Tuition
Online Toolkit
ePortfolio
Catalogs
Shuttle Tracker






About
Athletics
G


In [31]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)
doc_splits = text_splitter.split_documents(docs)


In [32]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embeddings,
    collection_name="agentic-rag-chroma"
)


<ipython-input-32-7492a69dacf0>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [33]:
retriever = vectorstore.as_retriever()


In [34]:
@tool
def get_mccoy_business_administration_context(question: str) -> str:
    """Retrieve context about Texas State University's McCoy College of Business Administration."""
    docs = retriever.get_relevant_documents(f"McCoy Business Administration {question}")
    if not docs:
        return "I could not find any relevant information about McCoy Business Administration."
    context = "\n".join([doc.page_content for doc in docs])
    return context


@tool
def get_fine_arts_context(question: str) -> str:
    """Retrieve context about Texas State University's Fine Arts and Communication programs."""
    docs = retriever.get_relevant_documents(f"Fine Arts and Communication {question}")
    if not docs:
        return "I could not find any relevant information about Fine Arts and Communication."
    context = "\n".join([doc.page_content for doc in docs])
    return context


@tool
def get_liberal_arts_context(question: str) -> str:
    """Retrieve context about Texas State University's Liberal Arts programs."""
    docs = retriever.get_relevant_documents(f"Liberal Arts {question}")
    if not docs:
        return "I could not find any relevant information about Liberal Arts."
    context = "\n".join([doc.page_content for doc in docs])
    return context


@tool
def get_science_and_engineering_context(question: str) -> str:
    """Retrieve context about Texas State University's Science and Engineering programs."""
    docs = retriever.get_relevant_documents(f"Science and Engineering {question}")
    if not docs:
        return "I could not find any relevant information about Science and Engineering."
    context = "\n".join([doc.page_content for doc in docs])
    return context


@tool
def get_health_professions_context(question: str) -> str:
    """Retrieve context about Texas State University's Health Professions programs."""
    docs = retriever.get_relevant_documents(f"Health Professions {question}")
    if not docs:
        return "I could not find any relevant information about Health Professions."
    context = "\n".join([doc.page_content for doc in docs])
    return context


@tool
def get_applied_arts_context(question: str) -> str:
    """Retrieve context about Texas State University's Applied Arts programs."""
    docs = retriever.get_relevant_documents(f"Applied Arts {question}")
    if not docs:
        return "I could not find any relevant information about Applied Arts."
    context = "\n".join([doc.page_content for doc in docs])
    return context


@tool
def get_education_context(question: str) -> str:
    """Retrieve context about Texas State University's Education programs."""
    docs = retriever.get_relevant_documents(f"Education {question}")
    if not docs:
        return "I could not find any relevant information about Education."
    context = "\n".join([doc.page_content for doc in docs])
    return context


In [35]:
tools = [
    get_mccoy_business_administration_context,
    get_fine_arts_context,
    get_liberal_arts_context,
    get_science_and_engineering_context,
    get_health_professions_context,
    get_applied_arts_context,
    get_education_context
]


In [37]:
system_prompt = """You are an AI academic advisor for Texas State University, specializing in providing detailed information about various undergraduate programs. You have access to the following tools: {tools}.

Use these tools to retrieve department-specific information as needed for Texas State’s McCoy Business Administration, Fine Arts and Communication, Liberal Arts, Science and Engineering, Health Professions, Applied Arts, Education, or IBM's involvement in the 2024 US Open.

When answering questions, follow this response format:

1. Analyze the question to determine the relevant department or topic.
2. Use the corresponding tool to retrieve context if necessary.
3. Provide a clear and detailed response based on the retrieved information or your existing knowledge.

Format:
{{ "action": "Tool Name", "action_input": "Tool Input" }}
Provide only ONE action per JSON block.

Example:
User: "Tell me about the Computer Information Systems major."

Response:
- Identify that this major is part of McCoy Business Administration.
- Use `get_mccoy_business_administration_context` to retrieve specific information.
- Respond with: "The Computer Information Systems major at Texas State University offers... [details from retrieved context]."

If no specific information is available, notify the user with a message such as:
"I couldn't find specific information on that topic, but I can help with general questions."

Your objective is to deliver comprehensive, department-focused answers that aid students in exploring and understanding Texas State University's programs and offerings.
"""


In [38]:
human_prompt = """Question: {input}
{agent_scratchpad}
"""


In [40]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field

In [43]:
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough

In [44]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", human_prompt),
])

# Finalize the prompt with tool names and descriptions
# from langchain.agents import render_tools_text

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([tool.name for tool in tools]),
)


In [45]:
memory = ConversationBufferMemory(memory_key="chat_history")


<ipython-input-45-1d4038f3f304>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [46]:
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # Using OpenAI functions for tool usage
    verbose=True,
    memory=memory,
    prompt=prompt,
)


<ipython-input-46-3f8124d6385b>:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent_executor = initialize_agent(


In [47]:
response = agent_executor.run("Can you provide details about the Computer Information Systems program at Texas State University, including available concentrations?")
print(response)




> Entering new AgentExecutor chain...

Invoking: `get_mccoy_business_administration_context` with `{'question': 'What are the details of the Computer Information Systems program at Texas State University, including available concentrations?'}`




<ipython-input-34-a3a95b7ed85f>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(f"McCoy Business Administration {question}")


Information Systems at Texas State University. More information about the Field of Study is available in the Academic Policies section of this catalog. If transferring additional business courses, please contact the McCoy College of Business
Information Systems at Texas State University. More information about the Field of Study is available in the Academic Policies section of this catalog. If transferring additional business courses, please contact the McCoy College of Business
Information Systems at Texas State University. More information about the Field of Study is available in the Academic Policies section of this catalog. If transferring additional business courses, please contact the McCoy College of Business
Information Systems at Texas State University. More information about the Field of Study is available in the Academic Policies section of this catalog. If transferring additional business courses, please contact the McCoy College of BusinessIt seems that I wasn't able to re

In [48]:
response = agent_executor.run(
    "What are the core courses, concentrations, and career opportunities available for students in the Computer Information Systems program at Texas State University?"
)
print(response)




> Entering new AgentExecutor chain...

Invoking: `get_mccoy_business_administration_context` with `{'question': 'What are the core courses, concentrations, and career opportunities available for students in the Computer Information Systems program?'}`


following common core of business courses or their equivalents as required by the McCoy College: 	  Course List        Code Title Hours    B A 1310Introduction to Business3 ISAN 1323Introduction to Microcomputer Applications3 ECO 2314Principles of
common body of knowledge in business, all students seeking the B.B.A. must complete the following common core of business courses or their equivalents as required by the McCoy College: 	  Course List        Code Title Hours    B A 1310Introduction
common body of knowledge in business, all students seeking the B.B.A. must complete the following common core of business courses or their equivalents as required by the McCoy College: 	  Course List        Code Title Hours    B A 1310Introduction


In [49]:
response = agent_executor.run("What is the capital of France?")
print(response)




> Entering new AgentExecutor chain...
The capital of France is Paris.

> Finished chain.
The capital of France is Paris.


In [50]:
print(memory.buffer)


Human: Can you provide details about the Computer Information Systems program at Texas State University, including available concentrations?
AI: It seems that I wasn't able to retrieve specific details about the Computer Information Systems program at Texas State University, including its concentrations. However, I can provide a general overview based on typical offerings in such programs.

The Computer Information Systems (CIS) program at Texas State University usually focuses on the intersection of business and technology, preparing students for careers in information technology and systems management. Common concentrations within CIS programs may include:

1. **Data Analytics**: Focuses on data analysis, data mining, and business intelligence.
2. **Cybersecurity**: Concentrates on protecting information systems from cyber threats.
3. **Software Development**: Emphasizes programming, software engineering, and application development.
4. **Network Administration**: Covers the manageme